In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2022-09-01 08:56:51,753: Credentials are already in use. The existing account in the session will be replaced.


In [3]:
def get_rightmost_one(gray_code_term, n):
    for i in range( (n-1), -1, -1):
        if(gray_code_term[i] == '1'):
            return i
    
    return 0


def get_second_rightmost_one(gray_code_term, rightmost_term, n):
    for i in range( (rightmost_term - 1), -1, -1):
        if(gray_code_term[i] == '1'):
            return i
        
    return 0

def reverse_string(string):
    #print("String: ", string)
    string = list(string)
    string.reverse()
    #print("String Reverse:" , string)
    #print("Return value: ", ''.join(string))
    return ''.join(string)

def modify_gray(gray_init, n):
    gray_final = []
    zero_element = ''
    for i in range(n):
        zero_element = zero_element + '0'
    
    gray_init.remove(zero_element)
    
    for i in gray_init:
        gray_final.append(reverse_string(i))
    
    return(gray_final)

def generate_gray(n):
    if( n == 1 ):
        return ['0','1']
    
    gray_n_minus_one = generate_gray( (n-1) )
    gray_n_minus_one_reverse = generate_gray( (n-1) )
    gray_n_minus_one_reverse.reverse()
    
    gray_n_minus_one = ['0' + x for x in gray_n_minus_one]
    gray_n_minus_one_reverse = ['1' + x for x in gray_n_minus_one_reverse]
    
    return gray_n_minus_one + gray_n_minus_one_reverse

def get_gray_diff(first, second, n):
    diff_index = 0
    for i in range(n):
        if(first[i] != second[i]):
            diff_index = i
            break
    
    return diff_index

def qncz(n):
    qncz = QuantumCircuit( (n + 1) )
    
    theta = np.pi / (2**(n - 1))
    
    gray_code_temp = generate_gray(n)
    gray_code = modify_gray(gray_code_temp, n)
    
    for i in range( len(gray_code) ):
        if (i == 0):
            current_index = 0
            qncz.cp(theta, 0, n)
            theta = -theta
            continue
        
        #print("Current Iteration: ", i, "Gray Code: ", gray_code[i])
    
        changing_index = get_gray_diff(gray_code[ (i - 1) ], gray_code[i], n)
        rightmost_index = get_rightmost_one(gray_code[i], n)
    
        #print("changing index: ", changing_index, " | rightmost index: ", rightmost_index)
        #print("checking")
    
        if(changing_index == rightmost_index):
            gray_code_copy = gray_code[i]
            target_index = rightmost_index
            source_index = get_second_rightmost_one(gray_code[i], rightmost_index, n)
            #print("* target index: ", target_index, " | source index: ", source_index)
        else:
            target_index = rightmost_index
            source_index = changing_index
            #print("target index: ", target_index, " | source index: ", source_index)
    
        qncz.cx(source_index, target_index)
        qncz.cp(theta, target_index, n)    
        theta = -theta

    qncz = qncz.to_gate()
    qncz.name = "  %iCZ" % n
    return qncz


In [ ]:
def decrement(n):
    qdec = QuantumCircuit(n)
    for i in range(n):
        qdec.x(i)
    
    for i in range((n-1), -1, -1):
        if(i == 0):
            qdec.x(i)
            continue
        if(i == 1):
            qdec.cx(0,1)
            continue
        if(i == 2):
            qdec.ccx()
            